# Support Vector Machine Model

## Importing libraries

In [1]:
import pickle
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import pandas as pd

## Reading in CSV data
Reading the dataset from a CSV into a DataFrame for use with the model.
Remove unnecessary data and split into data and labels.

In [2]:
music_df = pd.read_csv("extracted-features.csv")

## This will contain the data
data = music_df.iloc[:, 1:].values
## This will contain the labels corresponding to the data
labels = music_df.iloc[:, 0].values

## Cross-validate a SVM to find the best hyper-parameters

Use GridSearchCV to evaluate the optimal hyperparameters for the algorithm.

In [3]:
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size = 0.30)

svm_model = SVC(probability = True, kernel = "rbf")

param_grid = {"C": [0.1, 1, 10, 100, 1000],
              "gamma": ["scale", "auto"]}

scoring_metrics = {"Accuracy": "accuracy",
                   "AUC": "roc_auc_ovr",
                   "F1": "f1_macro"}

## roc_auc_ovr sets the score to use roc_auc in one vs rest mode
## StratifiedKFold is used by default for the cross validation
svm_grid_search_cv = GridSearchCV(svm_model,
                                  param_grid,
                                  scoring = scoring_metrics,
                                  refit = "AUC",
                                  cv = 5)

svm_grid_search_cv.fit(data_train, labels_train)

best_params = svm_grid_search_cv.best_params_

print(best_params)

svm_grid_predict = svm_grid_search_cv.predict(data_test)

print(classification_report(labels_test, svm_grid_predict))

## Best score is using the metric defined in refit
print("ROC AUC: " + str(svm_grid_search_cv.best_score_))

best_c_value = best_params["C"]
best_gamma = best_params["gamma"]

{'C': 1000, 'gamma': 'scale'}
              precision    recall  f1-score   support

       Blues       0.56      0.28      0.37        36
   Classical       0.57      0.83      0.68        24
     Country       0.14      0.09      0.11        33
       Disco       0.23      0.38      0.29        34
      HipHop       0.36      0.24      0.29        33
        Jazz       0.15      0.29      0.20        24
       Metal       0.42      0.65      0.51        23
         Pop       0.55      0.62      0.58        29
      Reggae       0.33      0.20      0.25        30
        Rock       0.14      0.06      0.08        34

    accuracy                           0.34       300
   macro avg       0.35      0.37      0.34       300
weighted avg       0.34      0.34      0.32       300

ROC AUC: 0.7905138149783104


## Produce SVM model

Produce a SVM model using the optimal hyperparameters and save to a Pickle file for use with Azure.

In [4]:
## svm_model = SVC(probability = True,
##                 class_weight = best_c_value,
##                 kernel = "rbf",
##                 gamma = best_gamma)
##
## svm_model.fit(data_train, labels_train)

with open("models/svm.pkl", mode = "wb") as model_file:
    pickle.dump(svm_grid_search_cv, model_file)